# Weaviate - Manage data

This file contains the code to manage Weaviate data

**Source: []()**

In [1]:
import os

In [2]:
os.getcwd()

'E:\\Accessories\\WebDevelopment\\Portfolio\\portfolio_admin_experience_django\\llm\\Weaviate'

In [5]:
import weaviate
from weaviate.connect import ConnectionParams
from weaviate.classes.init import AdditionalConfig, Timeout, Auth

# Connect to the client

In [6]:
weaviate_host = "localhost"
weaviate_port = 50050
weaviate_grpc_port = 50051
weaviate_scheme = "http"
weaviate_user = "admin@vip3rtech6069.com"
weaviate_key = "admin123"

weaviate_connection_params = ConnectionParams.from_params(
    http_host=weaviate_host,
    http_port=weaviate_port,
    http_secure=weaviate_scheme == "https",
    grpc_host=weaviate_host,
    grpc_port=weaviate_grpc_port,
    grpc_secure=weaviate_scheme == "https"
)
weaviate_auth_secret = Auth.api_key(weaviate_key)
weaviate_additional_config = AdditionalConfig(
    timeout=Timeout(init=30, query=60, insert=120),  # Values in seconds
)

In [9]:
client = weaviate.WeaviateClient(
    connection_params=weaviate_connection_params,
    auth_client_secret=weaviate_auth_secret,
    additional_config=weaviate_additional_config,
    skip_init_checks=True
)
client.connect()

In [10]:
client.is_ready()

True

# Manage collections



In [11]:
# Read all collections
client.collections.list_all()

{}